<a href="https://colab.research.google.com/github/raisa444-hub/raisa-15/blob/main/Timeseries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


# -------------------------------------------------------------
# Imports and configuration
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import statsmodels.api as sm
import shap
import joblib
import datetime

# Output directory
OUTDIR = "/content/project_outputs"
os.makedirs(OUTDIR, exist_ok=True)
print("Outputs saved to:", OUTDIR)

# -------------------------------------------------------------
# USER: set your data path here (CSV should include a timestamp column)
DATA_PATH = "/content/your_timeseries.csv"   # <-- change to your csv or upload file in Colab
TIMESTAMP_COL = "timestamp"                  # <-- change if different
TARGET_COLS = ["target"]                     # <-- the target column(s)
FEATURE_COLS = None                          # <-- None => use all except timestamp & targets

# -------------------------------------------------------------
# Helper: load dataset; if file missing, create a small synthetic example
def load_or_create_example(path):
    if os.path.exists(path):
        df = pd.read_csv(path)
        print("Loaded dataset:", path, "shape:", df.shape)
    else:
        print("Data not found at", path, "- creating synthetic dataset.")
        rng = pd.date_range("2020-01-01", periods=2000, freq="H")
        np.random.seed(0)
        x1 = np.sin(np.arange(len(rng))/24.0) + 0.1*np.random.randn(len(rng))
        x2 = 0.5*np.sin(np.arange(len(rng))/12.0) + 0.1*np.random.randn(len(rng))
        target = 2*x1 + 0.5*x2 + 0.2*np.random.randn(len(rng))
        df = pd.DataFrame({"timestamp": rng, "feat1": x1, "feat2": x2, "target": target})
    # ensure timestamp
    if TIMESTAMP_COL in df.columns:
        df[TIMESTAMP_COL] = pd.to_datetime(df[TIMESTAMP_COL])
        df = df.sort_values(TIMESTAMP_COL).reset_index(drop=True)
    return df

df = load_or_create_example(DATA_PATH)

# Determine feature columns if not set
if FEATURE_COLS is None:
    FEATURE_COLS = [c for c in df.columns if c not in ([TIMESTAMP_COL] + TARGET_COLS)]
print("Features:", FEATURE_COLS, "Targets:", TARGET_COLS)

# -------------------------------------------------------------
# 1) EDA (basic)
def eda_report(df, outdir=OUTDIR):
    print("Dataset head:")
    display(df.head())
    display(df.describe())
    # Plot target
    for t in TARGET_COLS:
        plt.figure(figsize=(10,2))
        plt.plot(df[TIMESTAMP_COL], df[t])
        plt.title("Target: "+t)
        plt.tight_layout()
        plt.savefig(os.path.join(outdir, f"plot_target_{t}.png"))
        plt.close()
eda_report(df)

# -------------------------------------------------------------
# 2) Preprocessing pipeline: imputation + scaling + time features
class PreprocessingPipeline:
    def __init__(self, feature_cols, target_cols, scaler=None):
        self.feature_cols = feature_cols
        self.target_cols = target_cols
        self.scaler = scaler if scaler else StandardScaler()
        self.fitted = False

    def add_time_features(self, df):
        if TIMESTAMP_COL in df.columns:
            df = df.copy()
            dt = pd.to_datetime(df[TIMESTAMP_COL])
            df["hour"] = dt.dt.hour
            df["dow"] = dt.dt.dayofweek
            df["month"] = dt.dt.month
        return df

    def fit_transform(self, df):
        df = df.copy()
        # Impute missing values: forward then backward
        df[self.feature_cols + self.target_cols] = df[self.feature_cols + self.target_cols].ffill().bfill()
        # Add time features
        df = self.add_time_features(df)
        # Columns to scale: features + time features
        scale_cols = self.feature_cols + ["hour","dow","month"]
        self.scaler.fit(df[scale_cols])
        df_scaled = df.copy()
        df_scaled[scale_cols] = self.scaler.transform(df[scale_cols])
        self.fitted = True
        return df_scaled

    def transform(self, df):
        if not self.fitted:
            raise RuntimeError("Pipeline not fitted")
        df = df.copy()
        df[self.feature_cols + self.target_cols] = df[self.feature_cols + self.target_cols].ffill().bfill()
        df = self.add_time_features(df)
        scale_cols = self.feature_cols + ["hour","dow","month"]
        df[scale_cols] = self.scaler.transform(df[scale_cols])
        return df

pipe = PreprocessingPipeline(FEATURE_COLS, TARGET_COLS)
df_processed = pipe.fit_transform(df)
joblib.dump(pipe, os.path.join(OUTDIR, "preprocessing_pipeline.joblib"))

# -------------------------------------------------------------
# 3) Unit tests for pipeline (simple checks)
def run_unit_tests(df_raw, df_processed):
    tests = {}
    # Test 1: no NaNs in processed features
    tests['no_nans'] = not df_processed[FEATURE_COLS + ["hour","dow","month"]].isna().any().any()
    # Test 2: scaler mean approximately 0 for scaled columns on training fit
    scaled_vals = df_processed[FEATURE_COLS + ["hour","dow","month"]]
    tests['scaled_mean_close_0'] = np.allclose(scaled_vals.mean(), 0, atol=1e-1)
    return tests

tests = run_unit_tests(df, df_processed)
print("Unit test results:", tests)
with open(os.path.join(OUTDIR, "unit_tests.json"), "w") as f:
    json.dump(tests, f)

# -------------------------------------------------------------
# 4) Create sequences (sliding window)
def make_sequences(df, feature_cols, target_cols, T_in=24, T_out=1, step=1):
    X, y = [], []
    vals = df[feature_cols + target_cols].values
    n_features = len(feature_cols) + 3  # if added hour,dow,month exist; we'll use df columns directly below to be safe
    total_cols = df[feature_cols + ["hour","dow","month"] + target_cols].values
    # Build using DataFrame columns to be consistent
    arr = df[feature_cols + ["hour","dow","month"] + target_cols].values
    in_cols = len(feature_cols) + 3
    for i in range(0, len(df) - T_in - T_out + 1, step):
        X.append(arr[i:i+T_in, :in_cols])
        y.append(arr[i+T_in:i+T_in+T_out, in_cols:])  # targets
    X = np.array(X)
    y = np.array(y)
    # If T_out==1, squeeze last dimension
    if T_out==1:
        y = y.squeeze(axis=1)
    return X, y

T_IN = 48   # use last 48 timesteps as input
T_OUT = 1
X, y = make_sequences(df_processed, FEATURE_COLS, TARGET_COLS, T_IN, T_OUT)
print("Sequences shapes:", X.shape, y.shape)

# Train/val/test split (temporal)
train_frac = 0.7
val_frac = 0.15
n = len(X)
i_train = int(n*train_frac)
i_val = int(n*(train_frac + val_frac))
X_train, y_train = X[:i_train], y[:i_train]
X_val, y_val = X[i_train:i_val], y[i_train:i_val]
X_test, y_test = X[i_val:], y[i_val:]
print("Train/Val/Test sizes:", X_train.shape[0], X_val.shape[0], X_test.shape[0])

# -------------------------------------------------------------
# 5) Baseline: SARIMAX (fit on training target series)
# For multivariate targets we would fit separate SARIMAX models per target; here's single-target example
def baseline_sarimax(df, target_col, train_end_idx, order=(1,0,1), seasonal_order=(0,0,0,0)):
    # df must be indexed by timestamp
    ss = df.set_index(TIMESTAMP_COL)[target_col]
    train = ss.iloc[:train_end_idx]
    model = sm.tsa.statespace.SARIMAX(train, order=order, seasonal_order=seasonal_order, enforce_stationarity=False, enforce_invertibility=False)
    res = model.fit(disp=False)
    return res

# Fit baseline
train_end_idx = i_train + T_IN  # approximate index in original df where training ends for series-based baseline
try:
    sarimax_res = baseline_sarimax(df, TARGET_COLS[0], train_end_idx)
    # Forecast on test period length
    steps = len(df) - train_end_idx
    sarimax_forecast = sarimax_res.get_forecast(steps=steps).predicted_mean
    # Align to test y indices and compute metrics for the last len(X_test) points
    sarimax_pred_for_test = sarimax_forecast[-len(X_test):]
    sarimax_rmse = np.sqrt(mean_squared_error(y_test.squeeze(), sarimax_pred_for_test))
    sarimax_mae = mean_absolute_error(y_test.squeeze(), sarimax_pred_for_test)
    print("SARIMAX RMSE:", sarimax_rmse, "MAE:", sarimax_mae)
    with open(os.path.join(OUTDIR, "baseline_metrics.txt"), "w") as f:
        f.write(f"SARIMAX RMSE: {sarimax_rmse}\nSARIMAX MAE: {sarimax_mae}\n")
except Exception as e:
    print("SARIMAX baseline failed (maybe small sample / exog mismatch). Error:", e)

# -------------------------------------------------------------
# 6) Build LSTM model
n_features = X_train.shape[2]
n_targets = 1 if len(TARGET_COLS)==1 else len(TARGET_COLS)

def build_lstm(T_in, n_features, n_targets, hidden_units=64, dropout=0.2):
    inp = layers.Input(shape=(T_in, n_features))
    x = layers.LSTM(hidden_units, return_sequences=True)(inp)
    x = layers.Dropout(dropout)(x)
    x = layers.LSTM(hidden_units//2)(x)
    x = layers.Dropout(dropout)(x)
    out = layers.Dense(n_targets)(x)  # one-step forecast
    model = models.Model(inputs=inp, outputs=out)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

model = build_lstm(T_IN, n_features, n_targets, hidden_units=128, dropout=0.2)
model.summary()

# -------------------------------------------------------------
# 7) Train with callbacks
checkpoint_path = os.path.join(OUTDIR, "best_model.h5")
cb = [
    callbacks.ModelCheckpoint(checkpoint_path, save_best_only=True, monitor='val_loss'),
    callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
]

history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32, callbacks=cb, verbose=2)
# Save final model and history
model.save(os.path.join(OUTDIR, "final_model.h5"))
pd.DataFrame(history.history).to_csv(os.path.join(OUTDIR, "training_history.csv"), index=False)

# -------------------------------------------------------------
# 8) Evaluate on test
# Fix: Pass custom_objects to models.load_model to resolve 'mse' lookup issue
model = models.load_model(checkpoint_path, custom_objects={'mse': tf.keras.losses.MeanSquaredError()})
y_pred = model.predict(X_test).squeeze()
rmse = np.sqrt(mean_squared_error(y_test.squeeze(), y_pred))
mae = mean_absolute_error(y_test.squeeze(), y_pred)
print("LSTM Test RMSE:", rmse, "MAE:", mae)
with open(os.path.join(OUTDIR, "lstm_metrics.txt"), "w") as f:
    f.write(f"LSTM RMSE: {rmse}\nLSTM MAE: {mae}\n")

# Plot predictions vs truth for first 200 points
plt.figure(figsize=(12,4))
plt.plot(y_test.squeeze()[:200], label="true")
plt.plot(y_pred[:200], label="pred")
plt.legend()
plt.title("Test predictions (first 200 points)")
plt.savefig(os.path.join(OUTDIR, "pred_vs_true.png"))
plt.close()

# Save predictions
np.save(os.path.join(OUTDIR, "y_test.npy"), y_test)
np.save(os.path.join(OUTDIR, "y_pred.npy"), y_pred)

# -------------------------------------------------------------
# 9) XAI with SHAP (explain model predictions using a small background sample)
# Use GradientExplainer if possible; otherwise KernelExplainer on a small subset (slower)
try:
    # Prepare an explainer with a background subset from train
    background = X_train[np.random.choice(len(X_train), size=min(200, len(X_train)), replace=False)]
    # For TF models, wrap predict function to produce output shape [N,]
    def f(pred_x):
        # Ensure output is always at least 1D, even for single sample, single target
        return np.atleast_1d(model.predict(pred_x).squeeze())
    # Use GradientExplainer or KernelExplainer based on model compatibility
    explainer = shap.GradientExplainer(model, background)  # may work for simple TF models
    test_sample = X_test[np.random.choice(len(X_test), size=min(100, len(X_test)), replace=False)]
    shap_values = explainer.shap_values(test_sample)  # shape: list or array

    # Flatten shap_values and test_sample for summary_plot
    if isinstance(shap_values, list):
        # Assuming shap_values is a list containing one array of shape (N_samples, T_in, n_features)
        shap_values_reshaped = shap_values[0].reshape(shap_values[0].shape[0], -1)
    else:
        # Assuming shap_values is an array of shape (N_samples, T_in, n_features)
        shap_values_reshaped = shap_values.reshape(shap_values.shape[0], -1)
    test_sample_reshaped = test_sample.reshape(test_sample.shape[0], -1)

    # Summary plot (global)
    shap.summary_plot(shap_values_reshaped, test_sample_reshaped, show=False)
    plt.savefig(os.path.join(OUTDIR, "shap_summary.png"), bbox_inches='tight')
    plt.close()
    print("SHAP explanation completed; saved shap_summary.png")
except Exception as e:
    print("SHAP GradientExplainer failed, falling back to KernelExplainer. Error:", e)
    # Fallback (much slower)
    background = X_train[np.random.choice(len(X_train), size=min(50, len(X_train)), replace=False)]
    # shap expects 2D arrays for kernel explainer, so we flatten timesteps*features per window
    def model_predict_flat(x_flat):
        # x_flat shape (N, T_in * n_features)
        x_reshaped = x_flat.reshape((-1, T_IN, n_features))
        # Ensure output is always at least 1D, even for single sample, single target
        return np.atleast_1d(model.predict(x_reshaped).squeeze())

    explainer = shap.KernelExplainer(model_predict_flat, background.reshape(background.shape[0], -1))
    test_sample = X_test[np.random.choice(len(X_test), size=min(30, len(X_test)), replace=False)]
    shap_values = explainer.shap_values(test_sample.reshape(test_sample.shape[0], -1), nsamples=100)

    # Flatten shap_values (which should be an array of shape (N_samples, T_in * n_features) or (N_samples, T_in * n_features, N_outputs))
    # It needs to be squeezed if there's a redundant dimension for single output, and then flattened.
    shap_values_reshaped = np.array(shap_values).squeeze().reshape(test_sample.shape[0], -1)
    test_sample_reshaped = test_sample.reshape(test_sample.shape[0], -1)

    shap.summary_plot(shap_values_reshaped, test_sample_reshaped, show=False)
    plt.savefig(os.path.join(OUTDIR, "shap_summary_kernel.png"), bbox_inches='tight')
    plt.close()
    print("SHAP KernelExplainer completed; saved shap_summary_kernel.png")

# -------------------------------------------------------------
# 10) Minimal textual report
report = {
    "data_shape": df.shape,
    "features": FEATURE_COLS,
    "targets": TARGET_COLS,
    "T_in": T_IN,
    "T_out": T_OUT,
    "train_size": X_train.shape[0],
    "val_size": X_val.shape[0],
    "test_size": X_test.shape[0],
    "lstm_rmse": float(rmse),
    "lstm_mae": float(mae)
}
with open(os.path.join(OUTDIR, "report.json"), "w") as f:
    json.dump(report, f, indent=2)
print("Report saved to", os.path.join(OUTDIR, "report.json"))

# -------------------------------------------------------------
# END
print("All done. Check output folder:", OUTDIR)

Outputs saved to: /content/project_outputs
Data not found at /content/your_timeseries.csv - creating synthetic dataset.
Features: ['feat1', 'feat2'] Targets: ['target']
Dataset head:


/tmp/ipython-input-1340789147.py:38: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  rng = pd.date_range("2020-01-01", periods=2000, freq="H")


,timestamp,feat1,feat2,target
0,2020-01-01 00:00:00,0.176405,-0.153292,0.398834
1,2020-01-01 01:00:00,0.081670,-0.129579,0.467291
2,2020-01-01 02:00:00,0.181111,0.087562,0.460220
3,2020-01-01 03:00:00,0.348764,0.027865,0.938750
4,2020-01-01 04:00:00,0.352652,0.155516,0.435396


,timestamp,feat1,feat2,target
count,2000,2000.000000,2000.000000,2000.000000
mean,2020-02-11 15:29:59.999999744,0.011139,0.002469,0.024426
min,2020-01-01 00:00:00,-1.233743,-0.765984,-2.849652
25%,2020-01-21 19:45:00,-0.683921,-0.331351,-1.352992
50%,2020-02-11 15:30:00,0.027712,0.005703,0.035422
75%,2020-03-03 11:15:00,0.709124,0.335100,1.359570
max,2020-03-24 07:00:00,1.275699,0.809281,2.766473
std,NaN,0.713145,0.365706,1.459928


Unit test results: {'no_nans': True, 'scaled_mean_close_0': True}
Sequences shapes: (1952, 48, 5) (1952, 1)
Train/Val/Test sizes: 1366 293 293


...

43/43 - 4s - 104ms/step - loss: 0.1041 - mae: 0.2558 - val_loss: 0.1027 - val_mae: 0.2566
Epoch 7/100
43/43 - 4s - 91ms/step - loss: 0.1069 - mae: 0.2606 - val_loss: 0.1337 - val_mae: 0.2895
Epoch 8/100
43/43 - 3s - 78ms/step - loss: 0.1058 - mae: 0.2591 - val_loss: 0.1033 - val_mae: 0.2557
Epoch 9/100
43/43 - 4s - 102ms/step - loss: 0.1044 - mae: 0.2594 - val_loss: 0.1082 - val_mae: 0.2613
Epoch 10/100
43/43 - 3s - 77ms/step - loss: 0.1065 - mae: 0.2597 - val_loss: 0.1099 - val_mae: 0.2659
Epoch 11/100
43/43 - 3s - 75ms/step - loss: 0.1049 - mae: 0.2590 - val_loss: 0.1055 - val_mae: 0.2563
Epoch 12/100
43/43 - 3s - 76ms/step - loss: 0.1011 - mae: 0.2544 - val_loss: 0.1106 - val_mae: 0.2657
Epoch 13/100
43/43 - 5s - 105ms/step - loss: 0.1034 - mae: 0.2553 - val_loss: 0.1083 - val_mae: 0.2593
Epoch 14/100
43/43 - 3s - 75ms/step - loss: 0.1054 - mae: 0.2596 - val_loss: 0.1080 - val_mae: 0.2590
Epoch 15/100


43/43 - 3s - 78ms/step - loss: 0.1034 - mae: 0.2582 - val_loss: 0.0976 - val_mae: 0.2521
Epoch 16/100
43/43 - 6s - 139ms/step - loss: 0.1066 - mae: 0.2601 - val_loss: 0.1063 - val_mae: 0.2592
Epoch 17/100
43/43 - 3s - 76ms/step - loss: 0.1023 - mae: 0.2547 - val_loss: 0.1142 - val_mae: 0.2674
Epoch 18/100
43/43 - 3s - 75ms/step - loss: 0.0976 - mae: 0.2496 - val_loss: 0.0984 - val_mae: 0.2508
Epoch 19/100
43/43 - 4s - 102ms/step - loss: 0.1000 - mae: 0.2528 - val_loss: 0.1100 - val_mae: 0.2639
Epoch 20/100
43/43 - 4s - 94ms/step - loss: 0.0998 - mae: 0.2507 - val_loss: 0.1033 - val_mae: 0.2551
Epoch 21/100
43/43 - 3s - 76ms/step - loss: 0.1023 - mae: 0.2551 - val_loss: 0.1122 - val_mae: 0.2662
Epoch 22/100
43/43 - 6s - 145ms/step - loss: 0.0980 - mae: 0.2506 - val_loss: 0.1083 - val_mae: 0.2603
Epoch 23/100
43/43 - 3s - 76ms/step - loss: 0.0972 - mae: 0.2487 - val_loss: 0.1078 - val_mae: 0.2589
Epoch 24/100
43/43 - 3s - 75ms/step - loss: 0.0970 - mae: 0.2476 - val_loss: 0.1133 - val_ma

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step
LSTM Test RMSE: 0.3258127966877472 MAE: 0.25466878933863096


/tmp/ipython-input-1340789147.py:280: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.
  shap.summary_plot(shap_values_reshaped, test_sample_reshaped, show=False)


SHAP explanation completed; saved shap_summary.png
Report saved to /content/project_outputs/report.json
All done. Check output folder: /content/project_outputs
